# 生成式预训练语言模型：理论与实战
深蓝学院 课程 
课程链接：https://www.shenlanxueyuan.com/course/620

作者 **黄佳**

## 第1步，构建一个简单的数据集

In [1]:
# 构建一个玩具数据集
corpus = [ "我喜欢吃苹果",
        "我喜欢吃香蕉",
        "她喜欢吃葡萄",
        "他不喜欢吃香蕉",
        "他喜欢吃苹果",
        "她喜欢吃草莓"] 

## 第2步：定义一个分词函数

In [2]:
# 定义一个分词函数，将文本转换为单字的列表
def tokenize(text):
    return [char for char in text]  # 将文本拆分为单字列表

# 对每个文本进行分词，并打印出对应的单字列表
print("单字列表:") 
for text in corpus:
    tokens = tokenize(text)
    print(tokens)

单字列表:
['我', '喜', '欢', '吃', '苹', '果']
['我', '喜', '欢', '吃', '香', '蕉']
['她', '喜', '欢', '吃', '葡', '萄']
['他', '不', '喜', '欢', '吃', '香', '蕉']
['他', '喜', '欢', '吃', '苹', '果']
['她', '喜', '欢', '吃', '草', '莓']


## 第3步：计算bigram词频

In [3]:
# 定义计算N-Gram词频的函数
from collections import defaultdict, Counter # 导入所需库
def count_ngrams(corpus, n):
    ngrams_count = defaultdict(Counter)  # 创建一个字典存储N-Gram计数
    for text in corpus:  # 遍历语料库中的每个文本
        tokens = tokenize(text)  # 对文本进行分词
        for i in range(len(tokens) - n + 1):  # 遍历分词结果生成N-Gram
            ngram = tuple(tokens[i:i+n])  # 创建一个N-Gram元组
            prefix = ngram[:-1]  # 获取N-Gram的前缀
            token = ngram[-1]  # 获取N-Gram的目标单字
            ngrams_count[prefix][token] += 1  # 更新N-Gram计数
    return ngrams_count
bigram_counts = count_ngrams(corpus, 2) # 计算Bigram词频
print("Bigram词频:") # 打印Bigram词频
for prefix, counts in bigram_counts.items():
    print("{}: {}".format("".join(prefix), dict(counts)))

Bigram词频:
我: {'喜': 2}
喜: {'欢': 6}
欢: {'吃': 6}
吃: {'苹': 2, '香': 2, '葡': 1, '草': 1}
苹: {'果': 2}
香: {'蕉': 2}
她: {'喜': 2}
葡: {'萄': 1}
他: {'不': 1, '喜': 1}
不: {'喜': 1}
草: {'莓': 1}


## 第4步：计算bigram概率


In [4]:
# 定义计算N-Gram概率的函数
def ngram_probabilities(ngram_counts):
    ngram_probs = defaultdict(Counter)  # 创建一个字典存储N-Gram概率
    for prefix, tokens_count in ngram_counts.items():  # 遍历N-Gram前缀
        total_count = sum(tokens_count.values())  # 计算当前前缀的N-Gram计数
        for token, count in tokens_count.items():  # 遍历每个前缀的N-Gram
            ngram_probs[prefix][token] = count / total_count  # 计算每个N-Gram概率
    return ngram_probs
bigram_probs = ngram_probabilities(bigram_counts) # 计算bigram概率
print("\nbigram概率:", bigram_probs) # 打印bigram概率


bigram概率: defaultdict(<class 'collections.Counter'>, {('我',): Counter({'喜': 1.0}), ('喜',): Counter({'欢': 1.0}), ('欢',): Counter({'吃': 1.0}), ('吃',): Counter({'苹': 0.3333333333333333, '香': 0.3333333333333333, '葡': 0.16666666666666666, '草': 0.16666666666666666}), ('苹',): Counter({'果': 1.0}), ('香',): Counter({'蕉': 1.0}), ('她',): Counter({'喜': 1.0}), ('葡',): Counter({'萄': 1.0}), ('他',): Counter({'不': 0.5, '喜': 0.5}), ('不',): Counter({'喜': 1.0}), ('草',): Counter({'莓': 1.0})})


## 第5步：定义生成下一个词的函数


In [5]:
# 定义生成下一个词的函数
def generate_next_token(prefix, ngram_probs):
    if not prefix in ngram_probs:  # 如果前缀不在N-Gram中，返回None
        return None
    next_token_probs = ngram_probs[prefix]  # 获取当前前缀对应的下一个词的概率
    next_token = max(next_token_probs, 
                     key=next_token_probs.get)  # 选择概率最大的词作为下一个词
    return next_token

## 第6步：生成连续文本

In [6]:
# 定义生成连续文本的函数
def generate_text(prefix, ngram_probs, n, length=6):
    tokens = list(prefix)  # 将前缀转换为字符列表
    for _ in range(length - len(prefix)):  # 根据指定长度生成文本 
        # 获取当前前缀对应的下一个词
        next_token = generate_next_token(tuple(tokens[-(n-1):]), ngram_probs) 
        if not next_token: # 如果下一个词为None，跳出循环
            break
        tokens.append(next_token) # 将下一个词添加到生成的文本中
    return "".join(tokens) # 将字符列表连接成字符串


In [7]:
# 输入一个前缀，生成文本
generated_text = generate_text("我", bigram_probs, 2)
print("\n生成的文本:", generated_text) # 打印生成的文本


生成的文本: 我喜欢吃苹果
